In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import datetime 
from sklearn.metrics.pairwise import cosine_similarity
krfont = {'family' : 'nanumgothic', 'weight' : 'bold', 'size'   : 10}
matplotlib.rc('font', **krfont)
plt.style.use('ggplot')
# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
#matplotlib.rcParams['axes.unicode_minus'] = Falseparse_dates=['date']

############

In [2]:
result = pd.read_csv('tot_weekly.csv')

In [3]:
result.columns

Index(['Date', 'FVJ', 'FMG', '철광석 Fines (Daily) [중국(수입가 CFR)] 현물USD/ton',
       'Bloomberg(DJ-UBS) Commodity Index1991=100',
       'Bloomberg Commodity Total Return1991=100',
       'Rogers International Commodities Index Metals1998.07.31=1000',
       '전기동 [LME] 현물USD/ton', '니켈 [LME] 현물USD/ton', '알루미늄 [LME] 현물USD/ton',
       '납 [LME] 현물USD/ton', 'BHP', 'RIO'],
      dtype='object')

In [4]:
result.columns = ['Date', 'FVJ', 'FMG', 'target',
       'Bloomberg(DJ-UBS) Commodity Index1991=100',
       'Bloomberg Commodity Total Return1991=100',
       'Rogers International Commodities Index Metals1998.07.31=1000',
       '전기동 [LME] 현물USD/ton', '니켈 [LME] 현물USD/ton', '알루미늄 [LME] 현물USD/ton',
       '납 [LME] 현물USD/ton', 'BHP', 'RIO']

In [5]:
result.head()

,Date,FVJ,FMG,target,Bloomberg(DJ-UBS) Commodity Index1991=100,Bloomberg Commodity Total Return1991=100,Rogers International Commodities Index Metals1998.07.31=1000,전기동 [LME] 현물USD/ton,니켈 [LME] 현물USD/ton,알루미늄 [LME] 현물USD/ton,납 [LME] 현물USD/ton,BHP,RIO
0,2008-12-05,1.256000,2.480,65.82,113.18660,226.7420,1295.272,3392.000000,9347.000000,1582.000000,1012.200000,35.846000,17.601500
1,2008-12-12,1.272000,2.478,69.38,111.38660,223.1360,1289.190,3131.200000,9700.000000,1476.600000,977.500000,39.687999,21.157000
2,2008-12-19,1.196000,2.396,74.15,112.76480,225.8960,1281.606,2961.500000,9717.000000,1433.100000,958.100000,42.610000,22.967500
3,2008-12-26,0.848000,1.775,76.43,111.43225,223.2300,1266.855,2843.666667,9723.333333,1493.333333,895.666667,39.727500,20.378125
4,2009-01-02,0.903333,1.930,72.54,115.98975,232.3625,1329.755,2907.000000,10783.750000,1463.125000,958.750000,42.660000,22.545625


## 뒤집기

In [6]:
result = result[::-1]
result=result.reset_index(drop=True)

In [7]:
result.head()

,Date,FVJ,FMG,target,Bloomberg(DJ-UBS) Commodity Index1991=100,Bloomberg Commodity Total Return1991=100,Rogers International Commodities Index Metals1998.07.31=1000,전기동 [LME] 현물USD/ton,니켈 [LME] 현물USD/ton,알루미늄 [LME] 현물USD/ton,납 [LME] 현물USD/ton,BHP,RIO
0,2018-07-27,2.8225,4.474,66.228,84.48646,174.182,2017.6020,6215.7,13541.0,2075.5,2136.7,49.949999,55.228000
1,2018-07-20,2.7520,4.440,64.852,83.10686,171.274,1999.6420,6081.5,13420.0,2083.4,2131.5,48.800000,54.058000
2,2018-07-13,2.7380,4.380,63.852,84.63404,174.356,2045.7940,6242.4,13897.0,2120.6,2230.0,49.950001,54.722000
3,2018-07-06,2.7200,4.342,64.054,85.84750,176.790,2074.2075,6472.8,14245.0,2132.8,2374.0,48.887500,54.122501
4,2018-06-29,2.8300,4.518,65.234,86.80800,178.696,2130.0600,6695.8,14792.0,2176.9,2418.2,49.162000,55.034000


In [8]:
from pandas import DataFrame
empty_df = DataFrame( columns = result.columns)
empty_df = empty_df.append([1,1,1,1,1])
empty_df = empty_df.iloc[:,:13]

/Users/choigirhyo/anaconda3/envs/tensorflow/lib/python3.5/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
  result = result.union(other)


In [9]:
result = pd.concat([empty_df,result])

In [10]:
result = result.reset_index()

In [11]:
feature1 = result[['Bloomberg(DJ-UBS) Commodity Index1991=100',
  'Bloomberg Commodity Total Return1991=100',
                  'FMG']][1:]

In [12]:
feature2 = result[['전기동 [LME] 현물USD/ton',
 '니켈 [LME] 현물USD/ton',
 'Rogers International Commodities Index Metals1998.07.31=1000','FVJ']][2:]



In [13]:
feature5 = result[['알루미늄 [LME] 현물USD/ton']][5:]

In [14]:
feature7 = result[['BHP','RIO']][7:]

In [15]:
feature1=feature1.reset_index(drop=True)
feature2=feature2.reset_index(drop=True)
feature5=feature5.reset_index(drop=True)
feature7=feature7.reset_index(drop=True)

In [16]:
print(result[['Date','target']].values.shape)
print(feature1.values.shape)
print(feature2.values.shape)
print(feature5.values.shape)
print(feature7.values.shape)

(509, 2)
(508, 3)
(507, 4)
(504, 1)
(502, 2)


In [17]:
result_table=pd.concat([result[['Date', 'target']],feature1,feature2,feature5,feature7],axis = 1)

In [200]:
print(result.index)
print(feature1.index)
print(feature2.index)
print(feature5.index)
print(feature7.index)

RangeIndex(start=0, stop=509, step=1)
RangeIndex(start=0, stop=508, step=1)
RangeIndex(start=0, stop=507, step=1)
RangeIndex(start=0, stop=504, step=1)
RangeIndex(start=0, stop=502, step=1)


In [20]:
result_table.head(8)

,Date,target,Bloomberg(DJ-UBS) Commodity Index1991=100,Bloomberg Commodity Total Return1991=100,FMG,전기동 [LME] 현물USD/ton,니켈 [LME] 현물USD/ton,Rogers International Commodities Index Metals1998.07.31=1000,FVJ,알루미늄 [LME] 현물USD/ton,BHP,RIO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2075.5,49.950001,54.722000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2083.4,48.887500,54.122501
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2120.6,49.162000,55.034000
3,NaN,NaN,NaN,NaN,NaN,6215.7,13541.0,2017.6020,2.8225,2132.8,48.238000,55.452000
4,NaN,NaN,84.48646,174.182,4.474,6081.5,13420.0,1999.6420,2.7520,2176.9,50.544000,58.378000
5,2018-07-27,66.228,83.10686,171.274,4.440,6242.4,13897.0,2045.7940,2.7380,2182.3,51.225999,59.143999
6,2018-07-20,64.852,84.63404,174.356,4.380,6472.8,14245.0,2074.2075,2.7200,2280.8,49.397500,56.692500
7,2018-07-13,63.852,85.84750,176.790,4.342,6695.8,14792.0,2130.0600,2.8300,2315.0,50.704001,58.091999


In [21]:
result_table.tail(8)

,Date,target,Bloomberg(DJ-UBS) Commodity Index1991=100,Bloomberg Commodity Total Return1991=100,FMG,전기동 [LME] 현물USD/ton,니켈 [LME] 현물USD/ton,Rogers International Commodities Index Metals1998.07.31=1000,FVJ,알루미늄 [LME] 현물USD/ton,BHP,RIO
501,2009-01-23,73.34,112.55740,225.4960,1.977,3254.900000,12238.000000,1406.766,1.106000,1433.1,35.846,17.6015
502,2009-01-16,74.09,119.64120,239.6800,2.134,2907.000000,10783.750000,1329.755,0.903333,1476.6,NaN,NaN
503,2009-01-09,73.64,115.98975,232.3625,1.930,2843.666667,9723.333333,1266.855,0.848000,1582.0,NaN,NaN
504,2009-01-02,72.54,111.43225,223.2300,1.775,2961.500000,9717.000000,1281.606,1.196000,NaN,NaN,NaN
505,2008-12-26,76.43,112.76480,225.8960,2.396,3131.200000,9700.000000,1289.190,1.272000,NaN,NaN,NaN
506,2008-12-19,74.15,111.38660,223.1360,2.478,3392.000000,9347.000000,1295.272,1.256000,NaN,NaN,NaN
507,2008-12-12,69.38,113.18660,226.7420,2.480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
508,2008-12-05,65.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 시점 미룬 타겟데이터 

In [22]:
a=result.target[1:]
a=a.reset_index(drop=True)
# 가격정보 1주전꺼를 feature로 만들자
result_table['target_1'] = a


In [23]:
a=result.target[2:]
a=a.reset_index(drop=True)
# 가격정보 1주전꺼를 feature로 만들자
result_table['target_2'] = a

In [24]:
a=result.target[3:]
a=a.reset_index(drop=True)
# 가격정보 1주전꺼를 feature로 만들자
result_table['target_3'] = a

In [25]:
a=result.target[4:]
a=a.reset_index(drop=True)
# 가격정보 1주전꺼를 feature로 만들자
result_table['target_4'] = a

In [26]:
a=result.target[5:]
a=a.reset_index(drop=True)
# 가격정보 1주전꺼를 feature로 만들자
result_table['target_5'] = a

## 뒤집기2 
### 왜 두번 뒤집혀있는지 모르겠습니다
### 고치면 앞뒤가 이상해질거 같아서 일단 놔두겠습니다

In [28]:
result_table = result_table[::-1]
result_table = result_table.reset_index(drop=True)

## 시점별로 데이터가 밀려있다

In [29]:
result_table.head(8)

,Date,target,Bloomberg(DJ-UBS) Commodity Index1991=100,Bloomberg Commodity Total Return1991=100,FMG,전기동 [LME] 현물USD/ton,니켈 [LME] 현물USD/ton,Rogers International Commodities Index Metals1998.07.31=1000,FVJ,알루미늄 [LME] 현물USD/ton,BHP,RIO,target_1,target_2,target_3,target_4,target_5
0,2008-12-05,65.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-12-12,69.38,113.18660,226.7420,2.480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.82,NaN,NaN,NaN,NaN
2,2008-12-19,74.15,111.38660,223.1360,2.478,3392.000000,9347.000000,1295.272,1.256000,NaN,NaN,NaN,69.38,65.82,NaN,NaN,NaN
3,2008-12-26,76.43,112.76480,225.8960,2.396,3131.200000,9700.000000,1289.190,1.272000,NaN,NaN,NaN,74.15,69.38,65.82,NaN,NaN
4,2009-01-02,72.54,111.43225,223.2300,1.775,2961.500000,9717.000000,1281.606,1.196000,NaN,NaN,NaN,76.43,74.15,69.38,65.82,NaN
5,2009-01-09,73.64,115.98975,232.3625,1.930,2843.666667,9723.333333,1266.855,0.848000,1582.0,NaN,NaN,72.54,76.43,74.15,69.38,65.82
6,2009-01-16,74.09,119.64120,239.6800,2.134,2907.000000,10783.750000,1329.755,0.903333,1476.6,NaN,NaN,73.64,72.54,76.43,74.15,69.38
7,2009-01-23,73.34,112.55740,225.4960,1.977,3254.900000,12238.000000,1406.766,1.106000,1433.1,35.846,17.6015,74.09,73.64,72.54,76.43,74.15


## 시점별로 데이터가 밀려있다

In [30]:
result_table.tail(8)

,Date,target,Bloomberg(DJ-UBS) Commodity Index1991=100,Bloomberg Commodity Total Return1991=100,FMG,전기동 [LME] 현물USD/ton,니켈 [LME] 현물USD/ton,Rogers International Commodities Index Metals1998.07.31=1000,FVJ,알루미늄 [LME] 현물USD/ton,BHP,RIO,target_1,target_2,target_3,target_4,target_5
501,2018-07-13,63.852,85.84750,176.790,4.342,6695.8,14792.0,2130.0600,2.8300,2315.0,50.704001,58.091999,64.054,65.234,66.130,67.3725,66.3025
502,2018-07-20,64.852,84.63404,174.356,4.380,6472.8,14245.0,2074.2075,2.7200,2280.8,49.397500,56.692500,63.852,64.054,65.234,66.1300,67.3725
503,2018-07-27,66.228,83.10686,171.274,4.440,6242.4,13897.0,2045.7940,2.7380,2182.3,51.225999,59.143999,64.852,63.852,64.054,65.2340,66.1300
504,NaN,NaN,84.48646,174.182,4.474,6081.5,13420.0,1999.6420,2.7520,2176.9,50.544000,58.378000,66.228,64.852,63.852,64.0540,65.2340
505,NaN,NaN,NaN,NaN,NaN,6215.7,13541.0,2017.6020,2.8225,2132.8,48.238000,55.452000,NaN,66.228,64.852,63.8520,64.0540
506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2120.6,49.162000,55.034000,NaN,NaN,66.228,64.8520,63.8520
507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2083.4,48.887500,54.122501,NaN,NaN,NaN,66.2280,64.8520
508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2075.5,49.950001,54.722000,NaN,NaN,NaN,NaN,66.2280


## 롤링

In [126]:
from pandas import DataFrame
moving_average = DataFrame()

In [127]:
aa =result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['target_1']
bb =result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['target_2']
cc =result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['target_3']
dd =result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['target_4']
ee =result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['target_5']

In [128]:
ma1 = result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['Bloomberg(DJ-UBS) Commodity Index1991=100']
ma2 = result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['Bloomberg Commodity Total Return1991=100']
ma3 = result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['FMG']
ma4 = result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['전기동 [LME] 현물USD/ton']
ma5 = result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['니켈 [LME] 현물USD/ton']
ma6 = result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['Rogers International Commodities Index Metals1998.07.31=1000']
ma7 = result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['FVJ']
ma8 = result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['알루미늄 [LME] 현물USD/ton']
ma9 = result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['BHP']
ma10 = result_table2.rolling(window=5).mean().fillna(result_table2.rolling(window=5).mean().mean())['RIO']

In [129]:
moving_average['Date'] = result_table['Date']
moving_average['target'] = result_table['target']

moving_average['target_1'] = aa
moving_average['ma1'] = ma1
moving_average['ma11'] = ma2
moving_average['ma111'] = ma3

moving_average['target_2']  = bb
moving_average['ma2'] = ma4
moving_average['ma22'] = ma5
moving_average['ma222'] = ma6
moving_average['ma2222'] = ma7

moving_average['target_3'] = cc
moving_average['target_4'] = dd


moving_average['target_5'] = ee
moving_average['ma5'] = ma8


moving_average['ma7'] = ma9
moving_average['ma77'] = ma10


In [130]:
moving_average = moving_average[7:]#moving average앞에 7개 제거

In [133]:
moving_average = moving_average.reset_index(drop = True)#moving average로 잘린 인덱스 다시 정리

In [134]:
moving_average.head()

,Date,target,target_1,ma1,ma11,ma111,target_2,ma2,ma22,ma222,ma2222,target_3,target_4,target_5,ma5,ma7,ma77
0,2009-01-23,73.34,74.170,114.47708,229.3329,2.04240,73.228,3019.653333,10432.416667,1314.8344,1.065067,71.664,102.218881,102.218881,1925.465503,58.362661,47.762396
1,2009-01-30,74.25,74.008,114.17887,228.7397,1.93720,74.170,3038.933333,10586.216667,1329.0384,1.005467,73.228,71.664000,102.218881,1925.465503,58.362661,47.762396
2,2009-02-06,78.83,73.572,114.19614,228.7789,1.92795,74.008,3087.153333,10837.816667,1339.8682,0.949147,74.170,73.228000,71.664000,1489.631667,58.362661,47.762396
3,2009-02-13,81.86,74.830,113.17239,226.7336,1.95795,73.572,3163.940000,11146.150000,1359.7052,0.946427,74.008,74.170000,73.228000,1481.091667,58.362661,47.762396
4,2009-02-20,74.98,76.474,111.26071,222.9116,2.04435,74.830,3240.840000,11273.600000,1370.9902,0.969360,73.572,74.008000,74.170000,1475.631667,40.106300,20.929950


In [135]:
moving_average.tail(6)

,Date,target,target_1,ma1,ma11,ma111,target_2,ma2,ma22,ma222,ma2222,target_3,target_4,target_5,ma5,ma7,ma77
496,2018-07-27,66.228,64.824400,85.468480,176.005600,4.450800,65.328500,6690.060000,14622.6000,2128.374800,2.823200,65.818600,66.065300,65.9505,2262.68,50.2815,57.7409
497,NaN,NaN,64.844000,84.976572,175.059600,4.430800,64.824400,6468.020000,14254.8000,2081.683200,2.782800,65.328500,65.818600,66.0653,2245.10,50.5951,58.1289
498,NaN,NaN,102.218881,118.022194,237.730232,4.365356,64.844000,6341.640000,13979.0000,2053.461100,2.772500,64.824400,65.328500,65.8186,2217.56,50.0219,57.5517
499,NaN,NaN,102.218881,118.022194,237.730232,4.365356,102.218881,6678.006783,15491.4925,2184.095404,3.091932,64.844000,64.824400,65.3285,2178.68,49.7135,56.9401
500,NaN,NaN,102.218881,118.022194,237.730232,4.365356,102.218881,6678.006783,15491.4925,2184.095404,3.091932,102.218881,64.844000,64.8244,2139.20,49.6115,56.4261
501,NaN,NaN,102.218881,118.022194,237.730232,4.365356,102.218881,6678.006783,15491.4925,2184.095404,3.091932,102.218881,102.218881,64.8440,2117.84,49.3563,55.5417


## 데이터 확인을 위해 Date를 남겨두자

In [55]:
#moving_average = moving_average.drop('Date',1) # 다 확인하고 Date 자르기

## Training_data = 모델훈련, Testing_data = 실제 제출할 때 사용할 데이터

In [181]:
Training_data = moving_average.dropna()
Testing_data = moving_average[moving_average['target'].isna()].drop(['target'],axis = 1) #model.predict()할 때 쓸 Test데이터

## Train,Valid데이터 나누기

In [145]:
def train_valid_split(Training_data,train_percent):
    train_data_num = round(len(Training_data)*train_percent)
    Train_data = Training_data[:train_data_num]
    Valid_data = Training_data[~Training_data.index.isin(Train_data.index)]
    TrainX = Train_data.drop(['target'],axis = 1)
    TrainY = Train_data['target']
    ValidX = Valid_data.drop(['target'],axis = 1)
    ValidY = Valid_data['target']
    return TrainX,TrainY,ValidX,ValidY

#확인해보기

In [192]:
TrainX,TrainY,ValidX,ValidY = train_valid_split(Training_data,0.7)

,Date,target_1,ma1,ma11,ma111,target_2,ma2,ma22,ma222,ma2222,target_3,target_4,target_5,ma5,ma7,ma77
0,2009-01-23,74.170000,114.47708,229.3329,2.04240,73.228000,3019.653333,10432.416667,1314.8344,1.065067,71.664000,102.218881,102.218881,1925.465503,58.362661,47.762396
1,2009-01-30,74.008000,114.17887,228.7397,1.93720,74.170000,3038.933333,10586.216667,1329.0384,1.005467,73.228000,71.664000,102.218881,1925.465503,58.362661,47.762396
2,2009-02-06,73.572000,114.19614,228.7789,1.92795,74.008000,3087.153333,10837.816667,1339.8682,0.949147,74.170000,73.228000,71.664000,1489.631667,58.362661,47.762396
3,2009-02-13,74.830000,113.17239,226.7336,1.95795,73.572000,3163.940000,11146.150000,1359.7052,0.946427,74.008000,74.170000,73.228000,1481.091667,58.362661,47.762396
4,2009-02-20,76.474000,111.26071,222.9116,2.04435,74.830000,3240.840000,11273.600000,1370.9902,0.969360,73.572000,74.008000,74.170000,1475.631667,40.106300,20.929950
5,2009-02-27,76.652000,109.39978,219.1919,2.21055,76.474000,3274.760000,10966.600000,1371.3290,1.009360,74.830000,73.572000,74.008000,1456.151667,42.003100,22.766950
6,2009-03-06,75.792000,108.12455,216.6475,2.38455,76.652000,3274.640000,10835.200000,1374.6790,1.100560,76.474000,74.830000,73.572000,1419.965000,42.099100,22.961150
7,2009-03-13,73.904000,106.72695,213.8595,2.53000,75.792000,3297.300000,10580.000000,1381.8272,1.179680,76.652000,76.474000,74.830000,1402.000000,41.095100,22.598025
8,2009-03-20,70.612000,105.52627,211.4667,2.59880,73.904000,3359.320000,10271.200000,1387.2832,1.262800,75.792000,76.652000,76.474000,1368.560000,41.015600,23.006900
9,2009-03-27,66.734000,105.40599,211.2387,2.54560,70.612000,3422.440000,9914.400000,1392.1740,1.307200,73.904000,75.792000,76.652000,1337.380000,40.679200,23.730975


In [195]:
print(Training_data.shape)
print(TrainX.shape)
print(ValidX.shape)


(497, 17)
(348, 16)
(149, 16)


## 주차별 TestX 뽑기 
### if - else 구문 안쓸 수 있는지 한번 고민해볼께요 ㅋㅋ 

In [189]:
def Test_What_Week(Testing_data,week):
    Testing_data = Testing_data.drop(['Date'],axis = 1)
    Testing_data.index = [1,2,3,4,5] #인덱스로 뽑아올려고 
    if week == 1:
        TestX = Testing_data.drop(['target_2','target_3','target_4','target_5'],axis = 1)
    elif week == 2:
        TestX = Testing_data.drop(['target_1','target_3','target_4','target_5','ma1','ma11','ma111'],axis = 1)
    elif week == 3:        
        TestX = Testing_data.drop(['target_1','target_2','target_4','target_5','ma1','ma11','ma111','ma2','ma22','ma222','ma2222'],axis = 1)
    elif week == 4:    
        TestX = Testing_data.drop(['target_1','target_2','target_3','target_5','ma1','ma11','ma111','ma2','ma22','ma222','ma2222'],axis = 1)
    elif week == 5:
        TestX = Testing_data.drop(['target_1','target_2','target_3','target_4','ma1','ma11','ma111','ma2','ma22','ma222','ma2222'],axis = 1)
    return TestX[TestX.index == week]


#확인해보기

In [191]:
Test_What_Week(Testing_data,1)

,target_1,ma1,ma11,ma111,ma2,ma22,ma222,ma2222,ma5,ma7,ma77
1,64.844,84.976572,175.0596,4.4308,6468.02,14254.8,2081.6832,2.7828,2245.1,50.5951,58.1289


In [196]:
Test_What_Week(Testing_data,2)

,target_2,ma2,ma22,ma222,ma2222,ma5,ma7,ma77
2,64.844,6341.64,13979.0,2053.4611,2.7725,2217.56,50.0219,57.5517


In [197]:
Test_What_Week(Testing_data,3)

,target_3,ma5,ma7,ma77
3,64.844,2178.68,49.7135,56.9401


In [198]:
Test_What_Week(Testing_data,4)

,target_4,ma5,ma7,ma77
4,64.844,2139.2,49.6115,56.4261


In [199]:
Test_What_Week(Testing_data,5)

,target_5,ma5,ma7,ma77
5,64.844,2117.84,49.3563,55.5417
